In [1]:
import logging
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text
import ast

/home/abdul-rehman/DISK-W/python/work-env/myvnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-20 01:58:12.900352: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 01:58:12.963173: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 01:58:13.195143: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 01:58:14.147018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find Ten

In [2]:
data = pd.read_csv('./processed/wiki-sql-2.csv')
# data = pd.read_csv('./processed/wiki_sql_full.csv')


In [3]:
def preprocess():

    # def formulate_schema(cols):
    #     index = []
    #     for idx, col in enumerate(cols):
    #         index.append(f'<col{idx + 1}> {col}')
    #     return index

    def combine_columns(row):
        return row['question'] + ' ' + ' '.join(row['table'])
    
    data['sql'] = data['sql'].apply(lambda x: (ast.literal_eval(x))['human_readable'])
    data['table'] = data['table'].apply(lambda x: (ast.literal_eval(x))['header'])
    # data['table'] = data['table'].apply(formulate_schema)

    data['question_header'] = data.apply(combine_columns, axis=1)
    data['question_header'] = data['question_header'].str.replace('/', ' ')



In [4]:
# preprocess()
data.question_header.iloc[0]

'Tell me what the notes are for South Australia  <col1> State territory <col2> Text background colour <col3> Format <col4> Current slogan <col5> Current series <col6> Notes'

In [5]:
model_path = 'ted_hrlr_translate_pt_en_converter'
# model = tf.saved_model.load(model_path)

model_name = 'ted_hrlr_translate_pt_en_converter'
# tf.keras.utils.get_file(
#     f'{model_name}.zip',
#     f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
#     cache_dir='.', cache_subdir='', extract=True
# )

tokenizers = tf.saved_model.load(model_name)

In [6]:
# encoded_questions = tokenizers.en.tokenize(data['question_header'].tolist())
# encoded_sql = tokenizers.en.tokenize(data['sql'].tolist())

# decoded_questions = tokenizers.en.detokenize(encoded_questions).numpy()
# decoded_sql = tokenizers.en.detokenize(encoded_sql).numpy()

In [7]:

# for i in range(2):
#     print(f"Original Question: {data['question_header'][i]}")
#     print(f"Tokenized Question: {encoded_questions[i]}")
#     print(f"Detokenized Question: {decoded_questions[i]}")

#     print(f"Original SQL: {data['sql'][i]}")
#     print(f"Tokenized SQL: {encoded_sql[i]}")
#     print(f"Detokenized SQL: {decoded_sql[i]}")

#     print("="*50)

In [8]:
# question_lengths = encoded_questions.row_lengths().numpy()
# sql_lengths = encoded_sql.row_lengths().numpy()

# # encoded_questions.row_lengths()

# for i in range(5):
#     print('Ques:', question_lengths[i],  '- SQL:', sql_lengths[i])

In [9]:

# all_lengths = np.concatenate([question_lengths, sql_lengths])

# # Plot the histogram
# plt.hist(all_lengths, np.linspace(0, 500, 101))
# plt.ylim(plt.ylim())
# max_length = max(all_lengths)

# # Add a vertical line at the maximum length
# plt.plot([max_length, max_length], plt.ylim())
# plt.title(f'Maximum tokens per example: {max_length}')
# plt.xlabel('Token Count')
# plt.ylabel('Number of Examples')
# plt.show()

In [10]:
MAX_TOKENS = 128

def prepare_batch(questions, sql_queries):
    # Tokenize questions and SQL queries
    questions_tokenized = tokenizers.en.tokenize(questions)
    sql_queries_tokenized = tokenizers.en.tokenize(sql_queries)
    
    # Trim tokenized sequences to MAX_TOKENS
    questions_tokenized = questions_tokenized[:, :MAX_TOKENS]
    sql_queries_tokenized = sql_queries_tokenized[:, :(MAX_TOKENS + 1)]  # Shift for labels
    
    # Convert to dense tensors
    questions_inputs = questions_tokenized.to_tensor()
    sql_inputs = sql_queries_tokenized[:, :-1].to_tensor()  # Drop the [END] tokens
    sql_labels = sql_queries_tokenized[:, 1:].to_tensor()  # Drop the [START] tokens
    
    return (questions_inputs, sql_inputs), sql_labels


In [11]:
# q = 'Tell me what the notes are for South Australia  State territory Text background colour Format Current slogan Current series Notes'
# s = 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'
# (q, s), s_label = prepare_batch([q], [s])
# print(q)
# print(s)
# print(s_label)


In [12]:
# # Same and shifted by 1
# print(s[:13])
# print(s_label[:13])

In [13]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [14]:
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

In [15]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(data, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

print("Train set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

Train set shape: (64523, 6)
Validation set shape: (8065, 6)
Test set shape: (8066, 6)


In [16]:
train_questions_data = train['question_header'].tolist()
train_sql_queries_data = train['sql'].tolist()

test_questions_data = test['question_header'].tolist()
test_sql_queries_data = test['sql'].tolist()

val_questions_data = val['question_header'].tolist()
val_sql_queries_data = val['sql'].tolist()
# Create TensorFlow Datasets

train_dataset = tf.data.Dataset.from_tensor_slices((train_questions_data, train_sql_queries_data))
test_dataset = tf.data.Dataset.from_tensor_slices((test_questions_data, test_sql_queries_data))
val_dataset = tf.data.Dataset.from_tensor_slices((val_questions_data, val_sql_queries_data))

In [17]:
train_batches = make_batches(train_dataset)
test_batches = make_batches(test_dataset)
val_batches = make_batches(val_dataset)

In [18]:
for (ques, sql), sql_labels in train_batches.take(1):
  break

print(ques.shape)
print(sql.shape)
print(sql_labels.shape)

(64, 107)
(64, 48)
(64, 48)


In [19]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [20]:
# #@title
# pos_encoding = positional_encoding(length=2048, depth=512)

# # Check the shape.
# print(pos_encoding.shape)

# # Plot the dimensions.
# plt.pcolormesh(pos_encoding.numpy().T, cmap='RdBu')
# plt.ylabel('Depth')
# plt.xlabel('Position')
# plt.colorbar()
# plt.show()

In [21]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = tf.cast(x, tf.int32)

    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


In [22]:
# VocabSize = tokenizers.en.get_vocab_size()
# print(VocabSize)

In [23]:
# embed_en = PositionalEmbedding(vocab_size=tokenizers.en.get_vocab_size(), d_model=512)


In [24]:
# ques = tf.cast(ques, dtype=tf.int32)
# embed_en(ques)

In [25]:
# embed_en(ques)

In [26]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [27]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)
   
    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [28]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [29]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [30]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x


In [31]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x
  
  # def get_config(self):
  #   config = super().get_config().copy()
  #   config.update({
  #       'd_model': self.d_model,
  #       'num_heads': self.num_heads,
  #       'dff': self.dff,
  #       "dropout_rate": self.dropout_rate
  #   })

In [32]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.
    
    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [33]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)
    
    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x
  
  # def get_config(self):
  #   config = super().get_config().copy()
  #   config.update({
  #       #'vocab_size': self.vocab_size,
  #       #'num_layers': self.num_layers,
  #       #'units': self.units,
  #       'd_model': self.d_model,
  #       'num_heads': self.num_heads,
  #       'dff': self.dff,
  #       "dropout_rate": self.dropout_rate
  #   })

In [34]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [35]:
from tensorflow import keras
@keras.utils.register_keras_serializable()
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1, **kwargs):
    super().__init__()
    self.num_layers = num_layers
    self.d_model = d_model
    self.num_heads = num_heads
    self.dff = dff
    self.input_vocab_size = input_vocab_size
    self.target_vocab_size = target_vocab_size
    self.dropout_rate = dropout_rate
        
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits
  
  def get_config(self):
    config = super().get_config().copy()
    config.update({
        'input_vocab_size': self.input_vocab_size,
        'target_vocab_size': self.target_vocab_size,
        'num_layers': self.num_layers,
        'd_model': self.d_model,
        'num_heads': self.num_heads,
        'dff': self.dff,
        "dropout_rate": self.dropout_rate
    })
    return config
  
  @classmethod
  def from_config(cls, config):
    # Extract required arguments from config
    num_layers = config.pop('num_layers')
    d_model = config.pop('d_model')
    num_heads = config.pop('num_heads')
    dff = config.pop('dff')
    input_vocab_size = config.pop('input_vocab_size')
    target_vocab_size = config.pop('target_vocab_size')
    dropout_rate = config.pop('dropout_rate', 0.1)  # Default value if not provided

    # Create an instance of Transformer with extracted arguments
    return cls(
      num_layers=num_layers, 
      d_model=d_model, 
      num_heads=num_heads,
      dff=dff, 
      input_vocab_size=input_vocab_size,
      target_vocab_size=target_vocab_size, 
      dropout_rate=dropout_rate, 
      **config
    )


In [36]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [37]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizers.en.get_vocab_size().numpy(),
    target_vocab_size=tokenizers.en.get_vocab_size().numpy(),
    dropout_rate=dropout_rate)

In [38]:
# output = transformer((ques, sql))

# print(ques.shape)
# print(sql.shape)
# print(output.shape)

In [39]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
  
  def get_config(self):
        return {
            'd_model': float(self.d_model),
            'warmup_steps': self.warmup_steps
        }

In [40]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [41]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [70]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy, 'accuracy'])

In [124]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath='./model-state/best_model.keras',  # Path to save the model file
    monitor='masked_accuracy',  # Monitor validation loss
    save_best_only=True,  # Save only the best model 
    # save_weights_only=True,  # Save only the model weights
    verbose=1  # Print verbose messages
)


In [125]:
history = transformer.fit(train_batches,
    epochs=5,
    validation_data=val_batches, 
    callbacks=[checkpoint_callback]
)

Epoch 1/5
1009/1009 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3824 - loss: 0.8198 - masked_accuracy: 0.8370
Epoch 1: masked_accuracy improved from -inf to 0.85320, saving model to ./model-state/best_model.keras
1009/1009 ━━━━━━━━━━━━━━━━━━━━ 3192s 3s/step - accuracy: 0.3824 - loss: 0.8197 - masked_accuracy: 0.8370 - val_accuracy: 0.4093 - val_loss: 0.5902 - val_masked_accuracy: 0.8812
Epoch 2/5
1009/1009 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4058 - loss: 0.4746 - masked_accuracy: 0.8978
Epoch 2: masked_accuracy improved from 0.85320 to 0.90594, saving model to ./model-state/best_model.keras
1009/1009 ━━━━━━━━━━━━━━━━━━━━ 3360s 3s/step - accuracy: 0.4058 - loss: 0.4745 - masked_accuracy: 0.8978 - val_accuracy: 0.4194 - val_loss: 0.3806 - val_masked_accuracy: 0.9221
Epoch 3/5
1009/1009 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4239 - loss: 0.3206 - masked_accuracy: 0.9292
Epoch 3: masked_accuracy improved from 0.90594 to 0.93363, saving model to ./model-state/best_model.k

In [73]:
transformer.save('./model-state/full-epoch-2.keras')

In [93]:
path = './model-state/full-epoch-2.keras'
new_model = tf.keras.models.load_model(
    path, 
    custom_objects={
        'Transformer': Transformer,
        'CustomSchedule': CustomSchedule,
        'masked_loss': masked_loss,
        'masked_accuracy': masked_accuracy,
    })


/home/abdul-rehman/DISK-W/python/work-env/myvnv/lib/python3.10/site-packages/keras/src/layers/layer.py:847: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/abdul-rehman/DISK-W/python/work-env/myvnv/lib/python3.10/site-packages/keras/src/layers/layer.py:847: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/abdul-rehman/DISK-W/python/work-env/myvnv/lib/python3.10/site-packages/keras/src/layers/layer.py:847: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask in

In [135]:
# print(history.history)

<Transformer name=transformer, built=True>

In [139]:
import pickle
import os

epoch = '(3,7)'

file_path = './model-state/historys.pkl'
tempH = history.history
historys = []

if os.path.exists(file_path):
    try:
        with open(file_path, 'rb') as f:
            historys = pickle.load(f)
    except (pickle.UnpicklingError, EOFError, TypeError):
        print(f"Error: Unable to unpickle file '{file_path}'. It may be corrupted or not in a valid pickle format.")

tempH['epoch'] = epoch
historys.append(tempH)

with open(file_path, 'wb') as f:
    pickle.dump(historys, f)


In [49]:
class Translator(tf.Module):
  def __init__(self, tokenizers, transformer):
    self.tokenizers = tokenizers
    self.transformer = transformer

  def __call__(self, sentence, max_length=MAX_TOKENS):
    # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
    assert isinstance(sentence, tf.Tensor)
    if len(sentence.shape) == 0:
      sentence = sentence[tf.newaxis]

    sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()

    encoder_input = sentence

    # As the output language is English, initialize the output with the
    # English `[START]` token.
    start_end = self.tokenizers.en.tokenize([''])[0]
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`.
    text = tokenizers.en.detokenize(output)[0]  # Shape: `()`.

    tokens = tokenizers.en.lookup(output)[0]

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    self.transformer([encoder_input, output[:,:-1]], training=False)
    attention_weights = self.transformer.decoder.last_attn_scores

    return text, tokens, attention_weights

In [122]:
translator = Translator(tokenizers, new_model)

In [51]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [52]:
print(data.question_header.iloc[4])
print(data.sql.iloc[4])

how many times is the fuel propulsion is cng? <col1> Order Year <col2> Manufacturer <col3> Model <col4> Fleet Series (Quantity) <col5> Powertrain (Engine Transmission) <col6> Fuel Propulsion
SELECT COUNT Fleet Series (Quantity) FROM table WHERE Fuel Propulsion = CNG


In [123]:
sentence = 'What is the format for South Australia? State territory Text background colour Format Current slogan Current series Notes'
ground_truth = 'SELECT Format FROM table WHERE State/territory = South Australia'

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

/home/abdul-rehman/DISK-W/python/work-env/myvnv/lib/python3.10/site-packages/keras/src/layers/layer.py:847: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/abdul-rehman/DISK-W/python/work-env/myvnv/lib/python3.10/site-packages/keras/src/layers/layer.py:847: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/abdul-rehman/DISK-W/python/work-env/myvnv/lib/python3.10/site-packages/keras/src/layers/layer.py:847: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask in

Input:         : What is the format for South Australia? State territory Text background colour Format Current slogan Current series Notes
Prediction     : select avg paris e / s / league from table where league of e ' s e ' sen & e ' sen & e ' s ) = 6 and position = 6 and gen
Ground truth   : SELECT Format FROM table WHERE State/territory = South Australia


In [121]:
import os
import pickle

epoch = '(,)'

file_path = './model-state/predictions.pkl'

if os.path.exists(file_path):
    try:
        with open(file_path, 'rb') as f:
            predictions = pickle.load(f)
    except (pickle.UnpicklingError, EOFError, TypeError):
        print(f"Error: Unable to unpickle file '{file_path}'. It may be corrupted or not in a valid pickle format.")

predictions.append({
    'epoch': epoch,
    'Input': sentence,
    'Prediction': translated_text,
    'Ground truth': ground_truth

})
# with open(file_path, 'wb') as f:
#     pickle.dump(predictions, f)



[{'Input': 'What is the format for South Australia? State territory Text background colour Format Current slogan Current series Notes',
  'Prediction': 'select lo ro ro from table where tddd = v and fm 7 and fm 2 = r',
  'Ground truth': 'SELECT Format FROM table WHERE State/territory = South Australia',
  'epoch': '(1,1)'},
 {'Input': 'What is the format for South Australia? State territory Text background colour Format Current slogan Current series Notes',
  'Prediction': "select avg paris e / s / league from table where league of e ' s e ' sen & e ' sen & e ' s ) = 6 and position = 6 and gen",
  'Ground truth': 'SELECT Format FROM table WHERE State/territory = South Australia',
  'epoch': '(2,2)'}]

In [101]:
sentence='Name the background colour for the Australian Capital Territory State territory Text background colour Format Current slogan Current series Notes'
ground_truth='SELECT Text/background colour FROM table WHERE State/territory = Australian Capital Territory'
translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

Input:         : Name the background colour for the Australian Capital Territory State territory Text background colour Format Current slogan Current series Notes
Prediction     : select en market from table where otherman = gamesress
Ground truth   : SELECT Text/background colour FROM table WHERE State/territory = Australian Capital Territory


In [59]:
sentence= 'how many times is the fuel propulsion is cng? Order Year Manufacturer Model Fleet Series (Quantity) Powertrain (Engine Transmission) Fuel Propulsion'
ground_truth='SELECT COUNT Fleet Series (Quantity) FROM table WHERE Fuel Propulsion = CNG'

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

NameError: name 'translator' is not defined